## Linear DioPhantine equations 

#### A Linear Diophantine Equation (in two variables) is an equation of the general form:
\\[ ax+by=c \tag{1}\\]
where \\( a,b,c \\) are given integers and \\( x,y\\) are unknown integers
###### so,we consider several classical problems on these equations:
- [Finding one solution](#1.Finding-a-solution)
- [finding all solutions](#2.Getting-all-solutions)
- [finding the number of solutions and the solutions themselves in a given interval](#3.Finding-the-number-of-solutions-and-the-solutions-in-a-given-interval)
- [finding a solution with minimum value of x+y](#4.Find-the-solution-with-minimum-value-of-\\(-x+y-\\))

###### Note: A degenerate case that need to be taken care of is when `a=b=0`. It is easy to see that we either have no solutions or infinitely many solutions, depending on whether c=0 or not.

### 1.Finding a solution
**Method**:
To find one solution of the Diophantine equation with 2 unknowns, you can use the Extended Euclidean algorithm. First, assume that a and b are non-negative. When we apply `Extended Euclidean algorithm` for `a` and `b`, we can find their greatest common divisor `g` and 2 numbers \\( x_g\\) and \\( y_g\\) such that:

\\[ ax_g+by_g=g \tag{2} \\] where `g = gcd(a,b)`
convert `eq(2)` into `eq(1)` by multiplying `c/g` on bothsides,we get
\\[ a.x_g.\left(\frac{c}{g}\right)+b.y_g.\left(\frac{c}{g}\right) = c \\]
so,compare with `eq(1)` \\[ x = x_g.\left(\frac{c}{g}\right),y=y_g.\left(\frac{c}{g}\right) \tag{3}\\]

In [54]:
from lib.BasicMath1 import gcdExtendedEuclidImprovedIterative

In [55]:
def isDegenerateCase(a,b,c):
    return True if a==0 and b==0 else False

In [56]:
def findAnySolution(a,b,c):
    if isDegenerateCase(a,b,c):
        return False
    (g,x_g,y_g) = gcdExtendedEuclidImprovedIterative(a,b)
    if c%g:
        return False
    x_0 = x_g*(c//g)
    y_0 = y_g*(c//g)
    if a<0:
        x_0 = -x_0
    if b<0:
        y_0 = -y_0
    return (g,x_0,y_0)

### 2.Getting all solutions

**Method**: From one solution \\( (x_0,y_0) \\), we can obtain all the solutions of the given equation.
- Consider \\( ax_0+by_0=c \text{ and } g=gcd(a,b)\\) 
- Now, we should see that adding b/g to x0, and, at the same time subtracting a/g from y0 will not break the equality:
\\[ a.\left(x_0+\frac{b}{g}\right) + b.\left(y_0-\frac{a}{g}\right) = a.x_0+b.y_0+a.\frac{b}{g}-b.\frac{a}{g}=c\\]
- Obviously, this process can be repeated again, so all the numbers of the form:
\\[ x=x_0+k.\frac{b}{g} \tag{4}\\]

\\[ y=y_0-k.\frac{a}{g} \tag{5}\\]
are solutions of the given Diophantine equation.since,it can have `k+1` integer pair solutions where `k=[1,infinity)`  

### 3.Finding the number of solutions and the solutions in a given interval
**Method**:so,it can have `infinte` solutions as per above section.Let's put some restrictions on the interval of `x` and `y`,and we will try to count and enumerate all the solutions.
- Let there be two intervals `[minx;maxx]` and `[miny;maxy]`.
- Note that if \\(a=0/b=0 \\),then the problem only has `one solution`.
- find any  \\( (x,y)\\).
- we will try to find two `1d` feasible regions on `x` when we shift `x` and `y` independently.
- **Region1 (shift x to range end points)**:
    - find \\( lx1,rx1 \\) through shifting \\( \frac{minx-x}{b},\frac{maxx-x}{b} \\) respectively (using `eq(4)`).
    - if \\( lx1 > maxx \\),then we get no solutions in the given Range.
- **Region2 (shift y to range end points)**
    - find \\( lx2,rx2 \\) through shifting \\( \frac{-(miny-y)}{a},\frac{-(maxy-y)}{a}\\) respectively (using `eq(5)`).
    - if \\( lx2 > maxy \\),then we get no solutions in the given Range.

- Now,All we need is \\( Region1 \cap Region2\\) end points \\( lx,rx \\).
- **No.of Solutions**: (from `eq(4)`)
\\[ \frac{(rx-lx)}{b}+1 \tag{6}\\]
- **Get All solutions**:
    - just need to iterate through \\( x = lx+k.\frac{b}{g} \\ \forall   k\geq 0,\text{ until } x = rx\\)
    - find `y` through `eq(1)`

In [52]:
def shiftSolution(x_0,y_0,a,b,k):
    return x_0+k*b, y_0-k*a

def findAllSolutions(a,b,c,minx,maxx,miny,maxy):
    if a==0 or b==0:
        return 1;
    if not findAnySolution(a,b,c):
        return 0;
    g,x,y = findAnySolution(a,b,c)
    
    # from eq(4),eq(5),simplifies shiftSolution equations
    a //= g;
    b //= g;
    
    sign_a = 1 if a>0 else -1
    sign_b = 1 if b>0 else -1
    
    x,y = shiftSolution(x,y,a,b,(minx-x)//b)
    if x < minx:
        x,y = shiftSolution(x,y,a,b,sign_b)
    if x > maxx:
        return 0;
    lx1 = x
    
    x,y = shiftSolution(x,y,a,b,(maxx-x)//b)
    if x > maxx:
        shiftSolution(x,y,a,b,-sign_b)
    rx1 = x
    
    x,y = shiftSolution(x,y,a,b,-(miny-y)//a)
    if y < miny:
        x,y = shiftSolution(x,y,a,b,-sign_a)
    if y > maxy:        
        return 0;
    lx2 = x
    
    x,y = shiftSolution(x,y,a,b,-(maxy-y)//a)
    if y > maxy:
        x,y  = shiftSolution(x,y,a,b,sign_a)
    rx2 = x
    
    if lx2 > rx2:
        temp = lx2;
        lx2 = rx2;
        rx2 = temp
    # Intersection
    lx = max(lx1,lx2)
    rx = min(rx1,rx2)
    
    if lx > rx:
        return 0;
    return (rx-lx)//abs(b)+1;   
    

### 4.Find the solution with minimum value of \\( x+y \\) 
-**Method**:
- using `eq(4)`,`eq(5)` ,we can figure out how \\( x+y \\) changes.
\\[ x+y = x_0+y_0+k.\left(\frac{b}{g}-\frac{a}{g}\right) = x_0+y_0+k.\frac{b-a}{g}\\]
- **a<b**:
    - select smallest possible value of `k`.
- **a>b**:
    - select largest possible value of `k`.
- **a=b**:
    - All solutions will have same sum `x+y`,so `k` doesn't matter.